In [ ]:
"""
@Description :   Digit recognition
@Author      :   Xubo Luo 
@Time        :   2024/04/04 12:41:38
"""

# 导入必要库

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 准备数据

In [ ]:
# Import the competition data
train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')

print(train_data.shape)
print(test_data.shape)

# Display rows and columns
train_data.head(10)

In [ ]:
# Drop labels
y = train_data['label']
train_data.drop("label", inplace=True, axis="columns")

In [ ]:
# Plot the first five digits along with the matrix conversion
sample1 = np.reshape(train_data.iloc[2, :].values, (28, 28))
plt.imshow(sample1)
plt.show()

sample2 = np.reshape(train_data.iloc[3, :].values, (28, 28))
plt.imshow(sample2)
plt.show()

sample3 = np.reshape(train_data.iloc[4, :].values, (28, 28))
plt.imshow(sample3)
plt.show()

sample4 = np.reshape(train_data.iloc[5, :].values, (28, 28))
plt.imshow(sample4)
plt.show()

sample5 = np.reshape(train_data.iloc[6, :].values, (28, 28))
plt.imshow(sample5)
plt.show()

In [ ]:
X = train_data.values.reshape((len(train_data), 28, 28))
X_test = test_data.values.reshape((len(test_data), 28, 28))

X = X / 255.0
X_test = X_test / 255.0

X = np.expand_dims(X, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42)

print(f"No of data train: {X_train.shape[0]}")
print(f"No of data val: {X_val.shape[0]}")
print(f"No of data test: {X_test.shape[0]}")

In [ ]:
def build_model(input_shape=(28, 28, 1)):
    model = tf.keras.Sequential(
            layers=[tf.keras.layers.Input(shape=input_shape),
                    tf.keras.layers.Conv2D(32, 3, activation='relu', data_format="channels_last"),
                    tf.keras.layers.Conv2D(32, 3, activation='relu', data_format="channels_last"),
                    tf.keras.layers.MaxPooling2D(),
                    tf.keras.layers.Dropout(0.25),
                    tf.keras.layers.Conv2D(32, 3, activation='relu', data_format="channels_last"),
                    tf.keras.layers.Conv2D(32, 3, activation='relu', data_format="channels_last"),
                    tf.keras.layers.MaxPooling2D(),
                    tf.keras.layers.Dropout(0.25),
                    tf.keras.layers.Flatten(),
                    tf.keras.layers.Dense(32, activation='relu'),
                    tf.keras.layers.Dense(64, activation='relu'),
                    tf.keras.layers.Dropout(0.25),
                    tf.keras.layers.Dense(128, activation='relu'),
                    tf.keras.layers.Dense(10, activation='softmax'),
                   ]
            )
    
    model.build()
    model.summary()
    
    return model

In [ ]:
model = build_model()

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="sparse_categorical_crossentropy",
              metrics=["acc"])

callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_acc",
                                                 factor=0.5,
                                                 patience=5,
                                                 min_lr=1e-3)

hist = model.fit(X_train, y_train, validation_data = (X_val, y_val),
                 epochs=20,
                 callbacks=[callbacks])

In [ ]:
pred = model.predict(X_test)
res = np.argmax(pred, axis=1)

In [ ]:
results = pd.Series(res, name="Label")
submission = pd.concat([pd.Series(range(1, X_test.shape[0] + 1), name="ImageId"), results], axis=1)
submission.to_csv("submission.csv", index=False)
print("Successfully saved as CSV file")